# Milestone 1

In this milestone, we are using the Figshare API to pull data and analyze it in upcoming milestones.

## Downloading the data from Figshare

In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

In [2]:
# Daily rainfall over NSW, Australia
# https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681
article_id = 14096681
# Metadata for the download
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "../data"

In [3]:
# List files in the associated dataset
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
files = data["files"]
files

[{'id': 26579150,
  'name': 'daily_rainfall_2014.png',
  'size': 58863,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e'},
 {'id': 26579171,
  'name': 'environment.yml',
  'size': 192,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34'},
 {'id': 26586554,
  'name': 'README.md',
  'size': 5422,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c'},
 {'id': 26766812,
  'name': 'data.zip',
  'size': 814041183,
  'is_link_only': False,
  'download_url': 'https://ndownloader.figshare.com/files/26766812',
  'supplied_md5': 'b517383f76e77bd03755a63a8f

In [4]:
# Retrieve `data.zip`
files_to_dl = ["data.zip"]
for f in files:
    if f["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(f["download_url"], f"{output_directory}/{f['name']}")

In [5]:
# Extract `data.zip`
output_zip_file = os.path.join(output_directory, "./data.zip")
with zipfile.ZipFile(output_zip_file, 'r') as f:
    f.extractall(output_directory)

## Combine CSV files

In [6]:
# Gather a list of files of CSV to merge
files = glob.glob(f'{output_directory}/*.csv')
files.remove(f'{output_directory}/observed_daily_rainfall_SYD.csv')
# files = files[0:1]
files

['../data/MPI-ESM-1-2-HAM_daily_rainfall_NSW.csv',
 '../data/AWI-ESM-1-1-LR_daily_rainfall_NSW.csv',
 '../data/NorESM2-LM_daily_rainfall_NSW.csv',
 '../data/ACCESS-CM2_daily_rainfall_NSW.csv',
 '../data/FGOALS-f3-L_daily_rainfall_NSW.csv',
 '../data/CMCC-CM2-HR4_daily_rainfall_NSW.csv',
 '../data/MRI-ESM2-0_daily_rainfall_NSW.csv',
 '../data/GFDL-CM4_daily_rainfall_NSW.csv',
 '../data/BCC-CSM2-MR_daily_rainfall_NSW.csv',
 '../data/EC-Earth3-Veg-LR_daily_rainfall_NSW.csv',
 '../data/CMCC-ESM2_daily_rainfall_NSW.csv',
 '../data/NESM3_daily_rainfall_NSW.csv',
 '../data/MPI-ESM1-2-LR_daily_rainfall_NSW.csv',
 '../data/ACCESS-ESM1-5_daily_rainfall_NSW.csv',
 '../data/FGOALS-g3_daily_rainfall_NSW.csv',
 '../data/INM-CM4-8_daily_rainfall_NSW.csv',
 '../data/MPI-ESM1-2-HR_daily_rainfall_NSW.csv',
 '../data/TaiESM1_daily_rainfall_NSW.csv',
 '../data/NorESM2-MM_daily_rainfall_NSW.csv',
 '../data/CMCC-CM2-SR5_daily_rainfall_NSW.csv',
 '../data/KIOST-ESM_daily_rainfall_NSW.csv',
 '../data/INM-CM5-

In [7]:
columns_to_merge = ["time", "lat_min", "lat_max", "lon_min", "lon_max", "rain (mm/day)"]

combined_path = f'{output_directory}/combined_data.csv'

In [8]:
%%time

# files = glob.glob('dailyrainfall/*.csv')
# df = pd.concat((pd.read_csv(files, index_col=0)
#                 .assign(model=re.findall("/([^_]*)", file)[0])
#                 for file in files)
#               )

# A Pythonic way (but not the most memory-efficient way) for merging the data
df = pd.concat((pd.read_csv(f, index_col=0, usecols=columns_to_merge)
                .assign(model=f[len(output_directory)+1:-len("_daily_rainfall_NSW.csv")])
                for f in files)
              )
df.to_csv(combined_path)

CPU times: user 3min 17s, sys: 12.9 s, total: 3min 30s
Wall time: 3min 30s


In [9]:
%%sh
du -sh ../data/combined_data.csv

5.6G	../data/combined_data.csv


In [10]:
print(df.shape)

(62467843, 6)


In [11]:
df.head()

,lat_min,lat_max,lon_min,lon_max,rain (mm/day),model
time,,,,,,
1889-01-01 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.244226e-13,MPI-ESM-1-2-HAM
1889-01-02 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.217326e-13,MPI-ESM-1-2-HAM
1889-01-03 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.498125e-13,MPI-ESM-1-2-HAM
1889-01-04 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.251282e-13,MPI-ESM-1-2-HAM
1889-01-05 12:00:00,-35.439867,-33.574619,141.5625,143.4375,4.270161e-13,MPI-ESM-1-2-HAM


> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken  |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-----------:|
> | Chen, Ziyi   |  OSX 13.2.1      | 32GB | M1 (10 processors)| YES    |  3min 30s   |
> | Guron, Mike  |                  |      |                   |        |             |
> | Raina, Roan  |                  |      |                   |        |             |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    | 10min 6secs |
> 
> Table 1: Time taken to combine the CSV files

## EDA

### Baseline

This is the baseline time needed to load the CSV file as-is.

In [12]:
%%time
df = pd.read_csv(f"{output_directory}/combined_data.csv")

CPU times: user 28.2 s, sys: 4.48 s, total: 32.7 s
Wall time: 34 s


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float64
 2   lat_max        float64
 3   lon_min        float64
 4   lon_max        float64
 5   rain (mm/day)  float64
 6   model          object 
dtypes: float64(5), object(2)
memory usage: 3.3+ GB


> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken    | Memory usage |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-------------:|:------------:|
> | Chen, Ziyi   |  OSX 13.2.1      | 32GB | M1 (10 processors)| YES    |  34 secs      |  3.3+ GB     |
> | Guron, Mike  |                  |      |                   |        |               |              |
> | Raina, Roan  |                  |      |                   |        |               |              |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    | 2 min 45 secs | 3.3+ GB      |
> 
> Table 2: Time taken to read the combined CSV (baseline)

#### Observations from Baseline

(WIP)

### Approach 1: Change the `dtype` of the data

We notice that by default it uses `float64` if we do not specify it. First, we try to see if switching to `float32` would make a smaller memory footprint, as well as a faster time.

In [14]:
%%time
df_float32 = pd.read_csv(f"{output_directory}/combined_data.csv", dtype={
    'lat_min': 'float32',
    'lat_max': 'float32',
    'lon_min': 'float32',
    'lon_max': 'float32',
    'rain (mm/day)': 'float32'
})

CPU times: user 28.1 s, sys: 3.04 s, total: 31.1 s
Wall time: 31.7 s


In [15]:
df_float32.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   time           object 
 1   lat_min        float32
 2   lat_max        float32
 3   lon_min        float32
 4   lon_max        float32
 5   rain (mm/day)  float32
 6   model          object 
dtypes: float32(5), object(2)
memory usage: 2.1+ GB


> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken    | Memory usage |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-------------:|:------------:|
> | Chen, Ziyi   |  OSX 13.2.1      | 32GB | M1 (10 processors)| YES    | 31.7 secs     | 2.1+ GB      |
> | Guron, Mike  |                  |      |                   |        |               |              |
> | Raina, Roan  |                  |      |                   |        |               |              |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    | 2 min 28 secs | 2.1+ GB      |
> 
> Table 3: Time taken to read the combined CSV (approach 1: use `float32` instead of `float64`)

#### Observations from Approach 1

(WIP)

### Approach 2: Load only column(s) we want

The dataset contains a number of columns that we may not need to use in one go. In this approach, we try to just load one column from the combined CSV file.

In [16]:
%%time
df_only_rain = pd.read_csv(f"{output_directory}/combined_data.csv", usecols=["rain (mm/day)"])

CPU times: user 15 s, sys: 1 s, total: 16 s
Wall time: 16.1 s


In [17]:
df_only_rain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62467843 entries, 0 to 62467842
Data columns (total 1 columns):
 #   Column         Dtype  
---  ------         -----  
 0   rain (mm/day)  float64
dtypes: float64(1)
memory usage: 476.6 MB


> | Team Member  | Operating System | RAM  | Processor         | Is SSD | Time taken    | Memory usage |
> |:------------:|:----------------:|:----:|:-----------------:|:------:|:-------------:|:------------:|
> | Chen, Ziyi   | OSX 13.2.1       | 32GB | M1 (10 processors)| YES    |   16.1 s      | 476.6 MB     |
> | Guron, Mike  |                  |      |                   |        |               |              |
> | Raina, Roan  |                  |      |                   |        |               |              |
> | Wong, Kelvin | Linux Mint 21    | 16GB | AMD Ryzen 5 3500U | YES    | 1 min 6 secs  | 476.6 MB     |
> 
> Table 4: Time taken to read the combined CSV (approach 2: just load `rain (mm/day)`)

#### Observations from Approach 2

(WIP)

## EDA in R

Here, we explore the EDA in R instead of Python. We try "exporting" our data frame as a Parquet file for processing in R.

In [18]:
%load_ext rpy2.ipython

In [19]:
%%time
df.to_parquet(f"{output_directory}/combined_data.parquet")

CPU times: user 10.2 s, sys: 2.99 s, total: 13.2 s
Wall time: 11.8 s


> Why we choose Parquet?
> 
> (WIP)

In [20]:
%%R
library(dplyr)
library(arrow)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘arrow’


R[write to console]: The following object is masked from ‘package:utils’:

    timestamp




In [21]:
%%R
r_parquet <- open_dataset("../data/combined_data.parquet")
r_df <- r_parquet |> collect()

In [22]:
%%time
%%R
r_df |> str()

tibble [62,467,843 × 7] (S3: tbl_df/tbl/data.frame)
 $ time         : chr [1:62467843] "1889-01-01 12:00:00" "1889-01-02 12:00:00" "1889-01-03 12:00:00" "1889-01-04 12:00:00" ...
 $ lat_min      : num [1:62467843] -35.4 -35.4 -35.4 -35.4 -35.4 ...
 $ lat_max      : num [1:62467843] -33.6 -33.6 -33.6 -33.6 -33.6 ...
 $ lon_min      : num [1:62467843] 142 142 142 142 142 ...
 $ lon_max      : num [1:62467843] 143 143 143 143 143 ...
 $ rain (mm/day): num [1:62467843] 4.24e-13 4.22e-13 4.50e-13 4.25e-13 4.27e-13 ...
 $ model        : chr [1:62467843] "MPI-ESM-1-2-HAM" "MPI-ESM-1-2-HAM" "MPI-ESM-1-2-HAM" "MPI-ESM-1-2-HAM" ...
CPU times: user 4.86 s, sys: 268 ms, total: 5.12 s
Wall time: 5.12 s


In [23]:
%%time
%%R
r_df |> summary()

In [ ]:
%%time
%%R
r_df |> head()

# A tibble: 6 × 7
  time                lat_min lat_max lon_min lon_max `rain (mm/day)` model     
  <chr>                 <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>     
1 1889-01-01 12:00:00   -35.4   -33.6    142.    143.        4.24e-13 MPI-ESM-1…
2 1889-01-02 12:00:00   -35.4   -33.6    142.    143.        4.22e-13 MPI-ESM-1…
3 1889-01-03 12:00:00   -35.4   -33.6    142.    143.        4.50e-13 MPI-ESM-1…
4 1889-01-04 12:00:00   -35.4   -33.6    142.    143.        4.25e-13 MPI-ESM-1…
5 1889-01-05 12:00:00   -35.4   -33.6    142.    143.        4.27e-13 MPI-ESM-1…
6 1889-01-06 12:00:00   -35.4   -33.6    142.    143.        4.20e-13 MPI-ESM-1…
CPU times: user 93.7 ms, sys: 6.88 ms, total: 101 ms
Wall time: 101 ms


In [ ]:
%%time
%%R
r_df |> tail()

# A tibble: 6 × 7
  time                lat_min lat_max lon_min lon_max `rain (mm/day)` model     
  <chr>                 <dbl>   <dbl>   <dbl>   <dbl>           <dbl> <chr>     
1 2014-12-26 12:00:00   -31.7   -29.8    153.    155.        4.91e- 1 MPI-ESM-1…
2 2014-12-27 12:00:00   -31.7   -29.8    153.    155.        3.22e- 4 MPI-ESM-1…
3 2014-12-28 12:00:00   -31.7   -29.8    153.    155.        4.61e-13 MPI-ESM-1…
4 2014-12-29 12:00:00   -31.7   -29.8    153.    155.        5.69e+ 0 MPI-ESM-1…
5 2014-12-30 12:00:00   -31.7   -29.8    153.    155.        1.23e+ 1 MPI-ESM-1…
6 2014-12-31 12:00:00   -31.7   -29.8    153.    155.        6.33e+ 0 MPI-ESM-1…
CPU times: user 18.1 ms, sys: 1.86 ms, total: 19.9 ms
Wall time: 19 ms
